# Proyek Analisis Data: Bike Sharing Dataset
- **Nama:** [Shadiqul Hakim]
- **Email:** [mmabdul339@gmail.com]
- **ID Dicoding:** [shadiqulbaikhati]

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1: Bagaimana pola penyewaan sepeda berdasarkan jam dalam sehari dan hari dalam seminggu? (Untuk mengidentifikasi waktu puncak penyewaan guna optimalisasi stok sepeda.)
- Pertanyaan 2: Faktor cuaca apa yang paling mempengaruhi jumlah penyewaan sepeda? (Untuk memprediksi permintaan berdasarkan kondisi lingkungan.)

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style for visualizations
sns.set(style='whitegrid')

## Data Wrangling

### Gathering Data

Kita load data dari day.csv dan hour.csv. Data harian untuk analisis agregat, data per jam untuk pola waktu.

In [ ]:
# Load datasets
day_df = pd.read_csv('day.csv')
hour_df = pd.read_csv('hour.csv')

# Display first few rows
print('Day Data:')
display(day_df.head())
print('Hour Data:')
display(hour_df.head())

**Insight:**
- Data harian mencakup 731 entri dengan variabel seperti season, temp, cnt (total rentals).
- Data per jam mencakup 17.379 entri, dengan tambahan 'hr' untuk jam, memungkinkan analisis waktu lebih detail.

### Assessing Data

Periksa tipe data, missing values, duplicates, dan statistik deskriptif.

In [ ]:
# Check info and missing values
print('Day Data Info:')
day_df.info()
print('\nMissing Values in Day:')
print(day_df.isnull().sum())
print('\nDuplicates in Day:', day_df.duplicated().sum())

print('\nHour Data Info:')
hour_df.info()
print('\nMissing Values in Hour:')
print(hour_df.isnull().sum())
print('\nDuplicates in Hour:', hour_df.duplicated().sum())

# Descriptive statistics
print('\nDay Descriptive Stats:')
display(day_df.describe())
print('\nHour Descriptive Stats:')
display(hour_df.describe())

**Insight:**
- Tidak ada missing values atau duplicates di kedua dataset.
- Tipe data 'dteday' perlu diubah ke datetime untuk analisis waktu. Variabel numerik seperti temp, hum normal (0-1 normalized).

### Cleaning Data

Ubah tipe data, hapus kolom tidak perlu (misalnya instant), dan handle outliers jika ada.

In [ ]:
# Convert dteday to datetime
day_df['dteday'] = pd.to_datetime(day_df['dteday'])
hour_df['dteday'] = pd.to_datetime(hour_df['dteday'])

# Drop unnecessary columns
day_df.drop('instant', axis=1, inplace=True)
hour_df.drop('instant', axis=1, inplace=True)

# Check for outliers in cnt (using IQR)
Q1 = day_df['cnt'].quantile(0.25)
Q3 = day_df['cnt'].quantile(0.75)
IQR = Q3 - Q1
day_df = day_df[(day_df['cnt'] >= Q1 - 1.5*IQR) & (day_df['cnt'] <= Q3 + 1.5*IQR)]

# Same for hour
Q1_h = hour_df['cnt'].quantile(0.25)
Q3_h = hour_df['cnt'].quantile(0.75)
IQR_h = Q3_h - Q1_h
hour_df = hour_df[(hour_df['cnt'] >= Q1_h - 1.5*IQR_h) & (hour_df['cnt'] <= Q3_h + 1.5*IQR_h)]

print('Data cleaned successfully.')

**Insight:**
- dteday diubah ke datetime untuk memudahkan filtering waktu.
- Outliers di cnt dihapus menggunakan IQR, memastikan data lebih representatif tanpa nilai ekstrem.

## Exploratory Data Analysis (EDA)

Explore distribusi, korelasi, dan grouping sederhana.

### Explore ...

In [ ]:
# Correlation heatmap for day data
plt.figure(figsize=(10,8))
sns.heatmap(day_df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Day Data')
plt.show()

# Group by season and weekday
season_group = day_df.groupby('season')['cnt'].mean()
weekday_group = day_df.groupby('weekday')['cnt'].mean()

print('Average Rentals by Season:', season_group)
print('Average Rentals by Weekday:', weekday_group)

**Insight:**
- Temp dan atemp berkorelasi positif tinggi dengan cnt (penyewaan), sementara hum dan windspeed negatif.
- Penyewaan tertinggi di musim 3 (fall) dan hari kerja (weekday 1-5).

## Visualization & Explanatory Analysis

### Pertanyaan 1: Bagaimana pola penyewaan sepeda berdasarkan jam dalam sehari dan hari dalam seminggu?

In [ ]:
# Hourly pattern
plt.figure(figsize=(12,6))
sns.lineplot(x='hr', y='cnt', data=hour_df, hue='workingday', ci=None)
plt.title('Hourly Bike Rentals: Working Day vs Non-Working Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Rentals')
plt.show()

# Weekly pattern
plt.figure(figsize=(8,5))
sns.barplot(x='weekday', y='cnt', data=day_df)
plt.title('Daily Bike Rentals by Weekday')
plt.xlabel('Weekday (0=Sunday, 6=Saturday)')
plt.ylabel('Average Rentals')
plt.show()

### Pertanyaan 2: Faktor cuaca apa yang paling mempengaruhi jumlah penyewaan sepeda?

In [ ]:
# Scatter plots for weather factors
fig, axes = plt.subplots(1, 3, figsize=(18,5))
sns.scatterplot(ax=axes[0], x='temp', y='cnt', data=day_df)
axes[0].set_title('Rentals vs Temperature')
sns.scatterplot(ax=axes[1], x='hum', y='cnt', data=day_df)
axes[1].set_title('Rentals vs Humidity')
sns.scatterplot(ax=axes[2], x='windspeed', y='cnt', data=day_df)
axes[2].set_title('Rentals vs Windspeed')
plt.show()

**Insight:**
- Pola jam: Puncak di jam 8 dan 17 pada hari kerja (komuter), lebih rata pada akhir pekan.
- Faktor cuaca: Temp positif mempengaruhi, hum dan windspeed negatif; temp paling kuat (korelasi >0.6).

## Analisis Lanjutan

Kita terapkan dua teknik:
1. Clustering manual (binning + grouping) berdasarkan faktor cuaca untuk segmentasi demand.
2. Adaptasi RFM analysis untuk segmentasi hari (Recency: hari sejak last high rental, Frequency: jumlah hari high rental, Monetary: total cnt).

In [ ]:
# Clustering Manual: Binning temp, hum, windspeed
day_df['temp_bin'] = pd.cut(day_df['temp'], bins=3, labels=['Low Temp', 'Med Temp', 'High Temp'])
day_df['hum_bin'] = pd.cut(day_df['hum'], bins=3, labels=['Low Hum', 'Med Hum', 'High Hum'])
day_df['wind_bin'] = pd.cut(day_df['windspeed'], bins=3, labels=['Low Wind', 'Med Wind', 'High Wind'])

# Manual grouping for demand cluster (integrasi windspeed)
def demand_cluster(row):
    if row['temp_bin'] == 'High Temp' and row['hum_bin'] in ['Low Hum', 'Med Hum'] and row['wind_bin'] == 'Low Wind':
        return 'High Demand'
    elif row['temp_bin'] == 'Med Temp' and row['wind_bin'] != 'High Wind':
        return 'Medium Demand'
    else:
        return 'Low Demand'

day_df['demand_cluster'] = day_df.apply(demand_cluster, axis=1)

# Group and visualize clustering
cluster_group = day_df.groupby('demand_cluster')['cnt'].mean()
plt.figure(figsize=(8,5))
sns.barplot(x=cluster_group.index, y=cluster_group.values)
plt.title('Average Rentals by Demand Cluster (Clustering Manual)')
plt.ylabel('Average Rentals')
plt.show()

print('Cluster Insights:', cluster_group)

# Adaptasi RFM Analysis untuk hari (segmentasi hari berdasarkan demand)
day_df = day_df.sort_values('dteday')
max_date = day_df['dteday'].max()
day_df['recency'] = (max_date - day_df['dteday']).dt.days  # Recency: hari sejak tanggal
high_rental_threshold = day_df['cnt'].quantile(0.75)  # High rental: atas 75%
day_df['is_high_rental'] = day_df['cnt'] > high_rental_threshold
rfm_df = day_df.groupby('yr').agg({  # Group by year for simplicity
    'recency': 'min',  # Recency: min days since last high rental
    'is_high_rental': 'sum',  # Frequency: count high rental days
    'cnt': 'sum'  # Monetary: total rentals
}).reset_index()
rfm_df.columns = ['year', 'recency', 'frequency', 'monetary']

# Binning RFM scores
rfm_df['r_score'] = pd.qcut(rfm_df['recency'], 4, labels=[4,3,2,1])  # Lower recency better
rfm_df['f_score'] = pd.qcut(rfm_df['frequency'], 4, labels=[1,2,3,4])
rfm_df['m_score'] = pd.qcut(rfm_df['monetary'], 4, labels=[1,2,3,4])
rfm_df['rfm_score'] = rfm_df['r_score'].astype(str) + rfm_df['f_score'].astype(str) + rfm_df['m_score'].astype(str)

print('RFM Analysis Results:')
display(rfm_df)

## Conclusion

- Conclusion pertanyaan 1: Pola penyewaan menunjukkan puncak pada jam pagi dan sore hari kerja, dengan hari kerja lebih tinggi daripada akhir pekan. Ini menunjukkan penggunaan utama untuk komuter.
- Conclusion pertanyaan 2: Suhu adalah faktor cuaca paling berpengaruh positif pada penyewaan, sementara kelembaban dan kecepatan angin menurunkannya. Bisnis bisa fokus pada promosi saat cuaca hangat.
- Conclusion analisis lanjutan: Clustering menunjukkan 'High Demand' pada hari dengan temp tinggi, hum rendah, wind rendah (average cnt tinggi). RFM adaptasi menunjukkan segmentasi tahun dengan score tinggi untuk tahun dengan high frequency/monetary.